<a href="https://colab.research.google.com/github/MorganPhillips02/Day-3-GitHub-Exercise/blob/main/CTM_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
severity_level = np.random.choice(["high", "medium", "low"], p=[0.2, 0.4, 0.4])
severity = np.random.choice(severity_levels[severity_level])
print(severity_level, '-', severity)

medium - HbA1c > 48


In [ ]:
def generate_blood_test_results():
  potassium = np.random.uniform(0, 6.5)
  eGFR = np.random.uniform(0, 50)
  PSA = np.random.uniform(0, 120)
  NewAF = np.random.choice([0, 1], p = [0.9, 0.1])
  Hb = np.random.uniform(0, 100)
  INR = np.random.uniform(0, 10)
  SuspiciousCXR = np.random.choice([0, 1], p = [0.05, 0.95])
  Troponin = np.random.uniform(0, 10)
  HbA1c = np.random.uniform(0, 10)
  LTFs = np.random.uniform(0, 10)
  Cholesterol = np.random.uniform(0, 10)
  Thyroid = np.random.uniform(0, 10)
  Anaemia =

  return potassium, eGFR, PSA, NewAF, Hb, INR, SuspiciousCXR, Troponin, HbA1c

  "high": ["K+ > 6", "eGFR < 15", "PSA > 100", "New AF on ECG", "Hb < 70", "INR > 8 on warfarin", "Highly suspicious CXR", "Troponin"],
    "medium": ["HbA1c > 48", "eGFR 30-45", "Mildly raised LTFs"],
    "low": ["Slightly raised cholesterol", "Stable thyroid levels", "Minor anaemia"]
}

In [4]:
import pandas as pd
import numpy as np

# Generate dataset with investigation features
num_samples = 1000
# Define task types
task_types = ["prescription request", "patient communication (via phone, NHS app, AccuRX)",
              "sick notes", "referral letters", "medical reports", "review results"]
# Define prescription-related variables
medication_types = {
    "high": ["Insulin", "Anti-epileptics", "Immunosuppressants"],
    "medium": ["Blood pressure medication", "Statins", "SSRIs"],
    "low": ["Vitamins", "Emollients", "Mild pain relief"]
}

request_types = {
    "high": ["Out-of-hours GP", "Pharmacist", "Phone"],
    "medium": ["Routine request via online", "NHS app"],
    "low": ["Routine scheduled repeats"]
}

# Define comorbidities
comorbidities = ["epilepsy", "historical heart attack / stroke", "diabetes", "mental health"]

# Define severity levels for test results
severity_levels = {
    "high": ["K+ > 6", "eGFR < 15", "PSA > 100", "New AF on ECG", "Hb < 70", "INR > 8 on warfarin", "Highly suspicious CXR", "Troponin"],
    "medium": ["HbA1c > 48", "eGFR 30-45", "Mildly raised LTFs"],
    "low": ["Slightly raised cholesterol", "Stable thyroid levels", "Minor anaemia"]
}

# Define investigation types
investigation_types = {
    "high": ["Troponin", "D-dimer", "eGFR drop", "Hb drop"],
    "medium": ["ECG", "Chest X-ray"],
    "low": ["Other tests"]
}

# Define test request urgency
test_request_urgency = {
    "high": ["Hospital-initiated urgent test"],
    "medium": ["Routine GP-initiated screening"],
    "low": ["Monitoring bloods with minor changes"]
}

# Function to generate prescription-related features
def generate_prescription_features():
    med_risk_level = np.random.choice(["high", "medium", "low"], p=[0.2, 0.4, 0.4])
    medication = np.random.choice(medication_types[med_risk_level])

    days_until_out = np.random.randint(-2, 10)
    polypharmacy = np.random.choice(["Yes", "No"], p=[0.3, 0.7])
    request_type = np.random.choice(["high", "medium", "low"], p=[0.2, 0.5, 0.3])
    request_source = np.random.choice(request_types[request_type])
    monitoring_required = np.random.choice([True, False], p=[0.3, 0.7])

    return medication, days_until_out, polypharmacy, request_source, monitoring_required

# Function to generate investigation-related features
def generate_investigation_features():
    severity_level = np.random.choice(["high", "medium", "low"], p=[0.2, 0.4, 0.4])
    severity = np.random.choice(severity_levels[severity_level])

    investigation_urgency_level = np.random.choice(["high", "medium", "low"], p=[0.3, 0.5, 0.2])
    investigation = np.random.choice(investigation_types[investigation_urgency_level])

    request_urgency_level = np.random.choice(["high", "medium", "low"], p=[0.3, 0.5, 0.2])
    request_urgency = np.random.choice(test_request_urgency[request_urgency_level])

    return severity, investigation, request_urgency

# Function to assign a priority score (0-100) instead of categorical labels
def assign_priority_score(task, comorbidities, time_since_requested, critical_result,
                          severity, investigation, request_urgency,
                          medication, days_until_out, polypharmacy, request_source, monitoring_required):
    score = 0

    # Increase score based on task type urgency
    if task == "review results":
        if severity in severity_levels["high"] or investigation in investigation_types["high"] or request_urgency in test_request_urgency["high"]:
            score += 80
        elif severity in severity_levels["medium"] or investigation in investigation_types["medium"] or request_urgency in test_request_urgency["medium"]:
            score += 50
        else:
            score += 20

    elif task == "prescription request":
        if medication in medication_types["high"] or days_until_out <= 1 or monitoring_required:
            score += 85
        elif medication in medication_types["medium"] or days_until_out <= 3 or polypharmacy == "Yes" or request_source in request_types["high"]:
            score += 55
        else:
            score += 25

    # Comorbidity influence
    for condition, flag in comorbidities.items():
        if flag:
            score += 5  # Small increase for each comorbidity

    # Increase priority score based on time waiting
    if time_since_requested > 72:
        score += 30
    elif time_since_requested > 48:
        score += 20

    # Cap the score at 100
    return min(score, 100)

# Generate dataset with numeric priority scores
data_full_priority = []
for _ in range(num_samples):
    task = np.random.choice(task_types)
    comorbidity_flags = {c: np.random.choice([0, 1], p=[0.85, 0.15]) for c in comorbidities}
    bmi = np.random.uniform(18, 40)
    time_since_requested = np.random.randint(1, 100)  # Hours
    estimated_duration = np.random.uniform(5, 120)  # Minutes

    # Initialize variables
    critical_result, severity, investigation, request_urgency = None, None, None, None
    medication, days_until_out, polypharmacy, request_source, monitoring_required = None, None, None, None, None

    if task == "review results":
        severity, investigation, request_urgency = generate_investigation_features()

    if task == "prescription request":
        medication, days_until_out, polypharmacy, request_source, monitoring_required = generate_prescription_features()

    priority_score = assign_priority_score(task, comorbidity_flags, time_since_requested, critical_result,
                                           severity, investigation, request_urgency,
                                           medication, days_until_out, polypharmacy, request_source, monitoring_required)

    # Assign category based on score
    if priority_score > 75:
        priority_category = "red"
    elif priority_score > 30:
        priority_category = "amber"
    else:
        priority_category = "green"

    row = [task] + list(comorbidity_flags.values()) + [bmi, time_since_requested, estimated_duration,
                                                       critical_result, severity, investigation, request_urgency,
                                                       medication, days_until_out, polypharmacy, request_source, monitoring_required,
                                                       priority_score, priority_category]
    data_full_priority.append(row)

# Create DataFrame with priority scores and categories
columns_priority = ["Task Type"] + comorbidities + ["BMI", "Time since task requested",
                   "Estimated task duration", "Critical result flag", "Severity of abnormality",
                   "Investigation Type", "Test Request Urgency", "Medication Type", "Days Until Out",
                   "Polypharmacy", "Request Type", "Monitoring Required", "Priority Score", "Priority Category"]

df_priority = pd.DataFrame(data_full_priority, columns=columns_priority)

# Show sample of updated dataset
df_priority.head()


,Task Type,epilepsy,historical heart attack / stroke,diabetes,mental health,BMI,Time since task requested,Estimated task duration,Critical result flag,Severity of abnormality,Investigation Type,Test Request Urgency,Medication Type,Days Until Out,Polypharmacy,Request Type,Monitoring Required,Priority Score,Priority Category
0,referral letters,0,0,0,0,33.198292,2,45.709260,None,None,None,None,None,NaN,None,None,None,0,green
1,review results,0,0,0,0,24.992394,62,77.677018,None,Stable thyroid levels,Chest X-ray,Routine GP-initiated screening,None,NaN,None,None,None,70,amber
2,referral letters,0,0,0,0,25.126152,38,116.090363,None,None,None,None,None,NaN,None,None,None,0,green
3,"patient communication (via phone, NHS app, Acc...",0,0,0,0,38.891704,23,30.383114,None,None,None,None,None,NaN,None,None,None,0,green
4,review results,0,0,0,0,19.991145,63,110.119743,None,eGFR 30-45,D-dimer,Monitoring bloods with minor changes,None,NaN,None,None,None,100,red


In [5]:
# prompt:  write me code to use scikit-learn to train a regression model. outcome is 'Priority Score' from df, all other columns excluding 'Priority Category' are input features. assess performance.

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Prepare the data
X = df_priority.drop(columns=['Priority Score', 'Priority Category'])
y = df_priority['Priority Score']

# Convert categorical features to numerical using one-hot encoding
X = pd.get_dummies(X, columns=['Task Type', 'Severity of abnormality', 'Investigation Type', 'Test Request Urgency', 'Medication Type', 'Request Type', 'Polypharmacy'], drop_first=True)

# Handle missing values (if any) - replace with the mean of the column
for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].mean(), inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestRegressor model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


<ipython-input-5-4da36bfa6433>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].mean(), inplace=True)
<ipython-input-5-4da36bfa6433>:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[col].fillna(X[col].mean(), inplace=True)


Mean Squared Error: 62.5274875
R-squared: 0.955237092137496
